# Lab 2 - Exploring Joining strategies

In this lab, we will investigate the joining of the property data from 2004-2015 to the water quality data.  First, we need to

1. Understand the big picture and determine the key(s) used to join tables.
2. Understand the relationship between tables (one-to-one; one-to-many; many-to-many), and
3. Pick a join type and investigate key mismatches.

## Problem 1 - Understanding the big picture and tables keys

![image.png](attachment:image.png)

**Task.**  The picture shown above provides a high-level view of the joining process. Remember that our goal is to explain changes in water quality in various lakes over time.  What do you think the structure of the final table should be?  One row per what? Describe some examples of the sort of columns you would want present in this file.

> <font color="orange"> Your thoughts here </font>
The structure of the final table should be a column with year, and a column with lake quality. The tables should be joined so that the data has one row per year, some columns I want present are the year, the lake, the quality overall for the lake.

## Problem 2 - Understanding the big picture and tables keys

**Tasks.**  Perform each of the following task.  Any task that requires data management should be performed separately in both `polars` and `pandas` using lazy data frames.

1. Use a lazy data frame to inspect the columns and a few rows of data.
2. Suggest the columns that will be used as keys to join the tables.
3. One set of keys are floating point numbers (which?).  Discuss the potential problems with using floating point numbers as keys.  Suggest possible remedies.
3. To understand the relationship (one-to-one; one-to-many; many-to-many) between tables, select the key columns, perform aggregation on each table to determine if there is one or many keys per row.
4. Based on the results of the last task, suggest a join type and justify your response.

In [24]:
from data.column_data import cols_to_keep
from data.columns import column_schema

In [25]:
import polars as pl
import polars.selectors as cs
from glob import glob
import re

In [27]:
glob('data/**/*.txt', recursive=True)

['data/MinneMUDAC_raw_files/mces_lakes_1999_2014.txt',
 'data/MinneMUDAC_raw_files/2015_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2003_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/mces_lakes_1999_2014_v2.txt',
 'data/MinneMUDAC_raw_files/2014_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/Parcel_Lake_Monitoring_Site_Xref.txt',
 'data/MinneMUDAC_raw_files/2008_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2006_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2007_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2002_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2011_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2013_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2002_metro_tax_parcels_v2.txt',
 'data/MinneMUDAC_raw_files/2005_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2010_metro_tax_parcels.txt',
 'data/MinneMUDAC_r

In [28]:
schema_override = {'latitude': pl.String(), 'longitude': pl.String()}

(mces_lakes := (
    pl.scan_csv(
        'data/MinneMUDAC_raw_files/mces_lakes_1999_2014_v2.txt',
        has_header=True,
        separator='\t',
        infer_schema_length=10000,
        schema_overrides=schema_override
    )
    .limit(100)
    .collect()
))

PROJECT_ID,DATA_SET_TITLE,LAKE_NAME,CITY,COUNTY,DNR_ID_Site_Number,MAJOR_WATERSHED,WATER_PLANNING_AUTHORITY,LAKE_SITE_NUMBER,START_DATE,START_HOURMIN24,END_DATE,END_HOURMIN24,SAMPLE_DEPTH_IN_METERS,Seasonal_Lake_Grade_RESULT,Seasonal_Lake_Grade_QUALIFIER,Seasonal_Lake_Grade_Units,Physical_Condition_RESULT,Physical_Condition_QUALIFIER,Physical_Condition_Units,Recreational_Suitability_RESULT,Recreational_Suitability_QUALIFIER,Recreational_Suitability_Units,Secchi_Depth_RESULT_SIGN,Secchi_Depth_RESULT,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude 7108,Citizen Assisted Monitoring Program (CAMP) for Lakes,Acorn Lake,Oakdale,Washington,…,0.011_duplicated_200,Approved_duplicated_141566,mg/L_duplicated_48254,-93.66768906_duplicated_78,44.88381717 7105_duplicated_78,Lakes Monitoring_duplicated_13814,Zumbra Lake_duplicated_79,Victoria_duplicated_878,Carver_duplicated_8436,10004100-01_duplicated_79,Mississippi River - Twin Cities_duplicated_15020,Minnehaha Creek WD_duplicated_2941,1_duplicated_60431,2002-10-14_duplicated_39,0:00_duplicated_27913,2002-10-14_duplicated_40,0:00_duplicated_27914,7_duplicated_624,_duplicated_432715,_duplicated_432716,_duplicated_432717,_duplicated_432718,_duplicated_432719,_duplicated_432720,_duplicated_432721,_duplicated_432722,_duplicated_432723,_duplicated_432724,_duplicated_432725,_duplicated_432726,m_duplicated_48255,~_duplicated_1634,0.012_duplicated_264,Approved_duplicated_141567,mg/L_duplicated_48255,-93.66768906_duplicated_79,44.88381717
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str


In [29]:
(parcel_lake_xref := (
    pl.scan_csv(
        'data/MinneMUDAC_raw_files/Parcel_Lake_Monitoring_Site_Xref.txt',
        has_header=True,
        separator='\t',
        infer_schema_length=10000
    )
    .limit(100)
    .collect()
))

Parcel_PIN,Monit_MAP_CODE1,Monit_SITE_CODE,Monit_LAKE_SITE,Distance_Parcel_Monitoring_Site_meters,Lake_Hydroid,Distance_Parcel_Lake_meters,centroid_long,centroid_lat,Parcel_pkey
str,str,i64,i64,f64,i64,f64,f64,f64,i64
null,"""19007900-01""",19007900,1,2815.49271,110517277058,2571.526792,-93.11451,44.94283,2163034
null,"""19007900-01""",19007900,1,2753.474688,110517277058,2515.373802,-93.11539,44.94234,2163035
null,"""19007900-01""",19007900,1,2748.810665,110517277058,2511.924959,-93.11556,44.94231,2163036
null,"""19007900-01""",19007900,1,2738.750056,110517277058,2502.99164,-93.11572,44.94223,2163037
null,"""19007900-01""",19007900,1,2700.541412,110517277058,2465.206234,-93.1158,44.94189,2163038
…,…,…,…,…,…,…,…,…,…
null,"""02004200-01""",2004200,1,6873.761858,110208412770,2270.762337,-93.2247,45.32081,1917269
null,"""02005300-01""",2005300,1,4921.073777,110208412730,4417.528519,-93.22211,45.30155,1917270
null,"""02005300-01""",2005300,1,4841.926764,110208412730,4291.816878,-93.21673,45.30076,1917271


## Problem 3 - Make join key indicator tables - Water Quality VS XREF

Now that you have identified the keys for joining the water quality file with the XREF, make an indicator column table as shown below.

![image.png](attachment:image.png)

Use this table to explore all mismatches

In [30]:
# Your code here
water_quality_keys = (
    pl.scan_csv(
        "data/MinneMUDAC_raw_files/mces_lakes_1999_2014.txt",
        has_header=True,
        separator='\t',
        infer_schema_length=10000,
        schema_overrides={'latitude': pl.String(), 'longitude': pl.String()}
    )
    .select(['DNR_ID_Site_Number', 'LAKE_NAME'])
    .unique()
    .with_columns(pl.lit(1).alias('water_quality'))
    .collect()
)
water_quality_keys

DNR_ID_Site_Number,LAKE_NAME,water_quality
str,str,i32


In [31]:
xref_keys = (
    pl.scan_csv(
        'data/MinneMUDAC_raw_files/Parcel_Lake_Monitoring_Site_Xref.txt',
        has_header=True,
        separator='\t'
    )
    .select(['Monit_MAP_CODE1'])
    .unique()
    .with_columns(pl.lit(1).alias('xref'))
    .collect()
)
xref_keys

Monit_MAP_CODE1,xref
str,i32
"""19005100-01""",1
"""19009500-01""",1
"""02000900-01""",1
"""27005700-01""",1
"""82004800-01""",1
…,…
"""82009900-02""",1
"""19008200-01""",1
"""82008000-01""",1


In [32]:
indicator_table = water_quality_keys.join(
    xref_keys,
    left_on="DNR_ID_Site_Number",
    right_on="Monit_MAP_CODE1",
    how="outer"
).fill_null(0)
indicator_table

/tmp/ipython-input-1273581701.py:1: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  indicator_table = water_quality_keys.join(


DNR_ID_Site_Number,LAKE_NAME,water_quality,Monit_MAP_CODE1,xref
str,str,i32,str,i32
null,null,0,"""19005100-01""",1
null,null,0,"""19009500-01""",1
null,null,0,"""02000900-01""",1
null,null,0,"""27005700-01""",1
null,null,0,"""82004800-01""",1
…,…,…,…,…
null,null,0,"""82009900-02""",1
null,null,0,"""19008200-01""",1
null,null,0,"""82008000-01""",1


In [33]:
indicator_table.filter(pl.col('xref') == 0)

DNR_ID_Site_Number,LAKE_NAME,water_quality,Monit_MAP_CODE1,xref
str,str,i32,str,i32


In [34]:
indicator_table.filter(pl.col('water_quality') == 0)

DNR_ID_Site_Number,LAKE_NAME,water_quality,Monit_MAP_CODE1,xref
str,str,i32,str,i32
null,null,0,"""19005100-01""",1
null,null,0,"""19009500-01""",1
null,null,0,"""02000900-01""",1
null,null,0,"""27005700-01""",1
null,null,0,"""82004800-01""",1
…,…,…,…,…
null,null,0,"""82009900-02""",1
null,null,0,"""19008200-01""",1
null,null,0,"""82008000-01""",1


## Problem 4 - Make an indicator column for the lat/long keys XREF VS parcel files.

To explore mismatches based on the lat/long keys, we will make another indicator table as shown below.

![image.png](attachment:image.png)

Again, use this table to explore all mismatches.

**Important.** Be sure to read the lat/long columns as strings!

In [35]:
# Your code here
import re
year_pattern = re.compile(r'(\d{4})_metro_tax_parcels')
getyear = lambda path: (m.group(1) if (m := year_pattern.search(path)) else None)

In [36]:
from glob import glob
# Find all parcel .txt files in data subfolders using glob
parcel_paths = [
    p
    for p in glob('data/**/*.txt', recursive=True)
    if 'parcel' in p
]
parcel_paths

['data/MinneMUDAC_raw_files/2015_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2003_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2014_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2008_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2006_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2007_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2002_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2011_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2013_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2002_metro_tax_parcels_v2.txt',
 'data/MinneMUDAC_raw_files/2005_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2010_metro_tax_parcels.txt',
 'data/MinneMUDAC_raw_files/2012_metro_tax_parcels.txt']

In [37]:
parcel_lat_long_tables = [
    (
        pl.scan_csv(
            path,
            has_header=True,
            separator="|",
            schema_overrides={"centroid_lat": pl.String(), "centroid_long": pl.String()}
        )
        .select(["centroid_lat", "centroid_long"])
        .unique()
        .with_columns(pl.lit(1).alias(str(getyear(path))))
        .collect()
    )
    for path in parcel_paths
]
parcel_lat_long_tables

[shape: (623_153, 3)
 ┌──────────────┬───────────────┬──────┐
 │ centroid_lat ┆ centroid_long ┆ 2015 │
 │ ---          ┆ ---           ┆ ---  │
 │ str          ┆ str           ┆ i32  │
 ╞══════════════╪═══════════════╪══════╡
 │ 44.8898      ┆ -93.56867     ┆ 1    │
 │ 45.24279     ┆ -93.21048     ┆ 1    │
 │ 45.13628     ┆ -93.2036      ┆ 1    │
 │ 45.41226     ┆ -93.47794     ┆ 1    │
 │ 45.06954     ┆ -93.23157     ┆ 1    │
 │ …            ┆ …             ┆ …    │
 │ 45.07186     ┆ -93.25322     ┆ 1    │
 │ 44.7964      ┆ -93.2239      ┆ 1    │
 │ 44.95793     ┆ -93.26036     ┆ 1    │
 │ 45.08        ┆ -93.26611     ┆ 1    │
 │ 45.04265     ┆ -93.29892     ┆ 1    │
 └──────────────┴───────────────┴──────┘,
 shape: (1_117_593, 3)
 ┌──────────────┬───────────────┬──────┐
 │ centroid_lat ┆ centroid_long ┆ 2003 │
 │ ---          ┆ ---           ┆ ---  │
 │ str          ┆ str           ┆ i32  │
 ╞══════════════╪═══════════════╪══════╡
 │ 45.02731     ┆ -93.47195     ┆ 1    │
 │ 44.71254 

In [38]:
join_next = lambda left, right: (
    left.join(
        right,
        on=['centroid_lat', 'centroid_long'],
        how='full'
    )
    .drop(cs.ends_with('_right'))
)
join_next

<function __main__.<lambda>(left, right)>

In [39]:
test_join = join_next(parcel_lat_long_tables[0], parcel_lat_long_tables[1])
test_join

centroid_lat,centroid_long,2015,2003
str,str,i32,i32
null,null,null,1
"""44.71254""","""-92.86241""",1,1
null,null,null,1
null,null,null,1
null,null,null,1
…,…,…,…
"""44.82633""","""-93.62458""",1,null
"""45.06131""","""-93.23437""",1,null
"""45.10877""","""-93.42588""",1,null


In [40]:
join_next(parcel_lat_long_tables[0], parcel_lat_long_tables[1])

centroid_lat,centroid_long,2015,2003
str,str,i32,i32
null,null,null,1
"""44.71254""","""-92.86241""",1,1
null,null,null,1
null,null,null,1
null,null,null,1
…,…,…,…
"""45.13087""","""-93.03296""",1,null
"""44.92502""","""-93.31644""",1,null
"""44.98845""","""-93.57151""",1,null


In [41]:
xref_lat_long_table = (
    pl.scan_csv(
        'data/MinneMUDAC_raw_files/Parcel_Lake_Monitoring_Site_Xref.txt',
        has_header=True,
        separator='\t',
        schema_overrides={'centroid_lat': pl.String(), 'centroid_long': pl.String()}
    )
    .select(['centroid_lat', 'centroid_long'])
    .unique()
    .with_columns([pl.lit(1).alias('xref')])
    .collect()
)
xref_lat_long_table

centroid_lat,centroid_long,xref
str,str,i32
"""44.67569""","""-93.17771""",1
"""44.6851""","""-93.1849""",1
"""45.17863""","""-93.32701""",1
"""44.7522""","""-93.31079""",1
"""45.01533""","""-93.38504""",1
…,…,…
"""44.69524""","""-92.75605""",1
"""44.66848""","""-93.25337""",1
"""45.12686""","""-93.24995""",1


In [42]:
from functools import reduce
# Reduce parcel indicator tables with join_next, starting from xref_latlong_table
final_indicator_table = (
    reduce(
        join_next,
        parcel_lat_long_tables,
        xref_lat_long_table
    )
    .filter(~(pl.col('centroid_lat').is_null() | pl.col('centroid_long').is_null()))
    .fill_null(0)
    .drop(['2002', '2003'])
    .with_columns(total_years = pl.sum_horizontal(cs.starts_with('20')))
)
final_indicator_table

centroid_lat,centroid_long,xref,2015,2014,2009,2008,2006,2007,2011,2013,2005,2004,2010,2012,total_years
str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""44.67569""","""-93.17771""",1,1,1,1,1,1,1,1,1,1,1,0,1,11
"""44.6851""","""-93.1849""",1,0,0,0,0,1,1,0,0,1,1,0,0,4
"""45.17863""","""-93.32701""",1,0,0,0,0,1,1,0,0,1,1,0,0,4
"""44.7522""","""-93.31079""",1,1,1,0,1,1,1,1,1,1,1,0,1,10
"""45.01533""","""-93.38504""",1,0,0,0,0,1,0,0,0,1,1,0,0,3
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""44.69524""","""-92.75605""",1,0,0,1,1,1,1,1,0,1,1,0,1,8
"""44.66848""","""-93.25337""",1,1,1,0,1,1,1,1,1,1,1,0,1,10
"""45.12686""","""-93.24995""",1,1,1,1,1,1,1,1,1,1,1,1,1,12


In [43]:
final_indicator_table.filter(pl.col('xref') == 0)

centroid_lat,centroid_long,xref,2015,2014,2009,2008,2006,2007,2011,2013,2005,2004,2010,2012,total_years
str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32


In [44]:
final_indicator_table.filter(pl.col('total_years') == 0)

centroid_lat,centroid_long,xref,2015,2014,2009,2008,2006,2007,2011,2013,2005,2004,2010,2012,total_years
str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""45.059""","""-92.98129""",1,0,0,0,0,0,0,0,0,0,0,0,0,0
"""45.22142""","""-93.22728""",1,0,0,0,0,0,0,0,0,0,0,0,0,0
"""45.14041""","""-92.79815""",1,0,0,0,0,0,0,0,0,0,0,0,0,0
"""44.92688""","""-93.38634""",1,0,0,0,0,0,0,0,0,0,0,0,0,0
"""45.03322""","""-92.82467""",1,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""44.96764""","""-93.35587""",1,0,0,0,0,0,0,0,0,0,0,0,0,0
"""44.69202""","""-93.44948""",1,0,0,0,0,0,0,0,0,0,0,0,0,0
"""45.01544""","""-93.25757""",1,0,0,0,0,0,0,0,0,0,0,0,0,0
